In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os


In [9]:
# Tentukan path ke dataset
dataset_dir = './dataset'

In [10]:
# Membaca dataset
categories = os.listdir(dataset_dir)  # List semua folder di dalam dataset
data_count = {}

# Menghitung jumlah gambar per kategori
for category in categories:
    category_path = os.path.join(dataset_dir, category)
    if os.path.isdir(category_path):  # Pastikan hanya membaca folder
        data_count[category] = len(os.listdir(category_path))

# Menampilkan hasil
print("Jumlah gambar per kategori:")
for category, count in data_count.items():
    print(f"- {category}: {count} gambar")

# Validasi jumlah total dataset
total_images = sum(data_count.values())
print(f"\nTotal jumlah gambar dalam dataset: {total_images}")

Jumlah gambar per kategori:
- cardboard: 403 gambar
- metal: 410 gambar
- paper: 594 gambar
- plastic: 482 gambar
- trash: 137 gambar

Total jumlah gambar dalam dataset: 2026


In [ ]:
# Membaca dataset dan membaginya menjadi training dan validation
# ImageDataGenerator digunakan untuk pemrosesan gambar
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load data training dan validation
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1623 images belonging to 5 classes.
Found 403 images belonging to 5 classes.


In [ ]:
# Menambahkan augmentasi data dengan variasi lebih besar
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,  # Memperbesar rotasi hingga 50 derajat
    width_shift_range=0.3,  # Memperbesar pergeseran horizontal
    height_shift_range=0.3,  # Memperbesar pergeseran vertikal
    shear_range=0.3,  # Meningkatkan shear (kecondongan)
    zoom_range=0.3,  # Memperbesar zoom range
    horizontal_flip=True,  # Menambahkan flip horizontal
    fill_mode='nearest',  # Metode pengisian setelah transformasi
    validation_split=0.2  # Menentukan subset data untuk validasi
)

# Apply augmentasi pada data training
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Apply augmentasi pada data validasi
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1623 images belonging to 5 classes.
Found 403 images belonging to 5 classes.


In [13]:
# Membuat model CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(4, activation='softmax')  # Output layer for 4 classes
])

# Menyusun model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [14]:
# Melatih model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/30


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_2" is incompatible with the layer: expected axis -1 of input shape to have value 36992, but received input with shape (None, 86528)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)
  • training=True
  • mask=None

In [ ]:
# Visualisasi hasil training
plt.figure(figsize=(12, 4))

# Grafik Akurasi
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Akurasi Training')
plt.plot(history.history['val_accuracy'], label='Akurasi Validation')
plt.title('Akurasi Training dan Validation')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()

# Grafik Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss Training')
plt.plot(history.history['val_loss'], label='Loss Validation')
plt.title('Loss Training dan Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
# Evaluasi model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


In [ ]:
# Menyimpan model
model.save('sampah_classifier_model-index4.h5')